<a href="https://colab.research.google.com/github/Karanuderani07/ML_LEARNING/blob/main/Cross_Validation_Bias_Variance_5exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

from google.colab import files
uploaded = files.upload()


file_name = next(iter(uploaded))
df = pd.read_csv(file_name)

Saving weatherHistory.csv to weatherHistory (2).csv


In [9]:
display(df.head())

,Formatted_Date,Summary,Precip_Type,Temperature (C),Apparent_Temperature_C,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251,15.8263,0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259,15.8263,0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204,14.9569,0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269,15.8263,0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259,15.8263,0,1016.51,Partly cloudy throughout the day.


In [11]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

df['Formatted_Date'] = pd.to_datetime(df['Formatted_Date'], utc=True)

df['Hour'] = df['Formatted_Date'].dt.hour
df['Day'] = df['Formatted_Date'].dt.day

label_encoder = LabelEncoder()
df['Precip Type Encoded'] = label_encoder.fit_transform(df['Precip_Type'])

X = df[['Hour', 'Day', 'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)',
        'Visibility (km)', 'Loud Cover', 'Pressure (millibars)', 'Temperature (C)']]
y = df['Precip Type Encoded']

In [12]:
df.head()

,Formatted_Date,Summary,Precip_Type,Temperature (C),Apparent_Temperature_C,Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,Hour,Day,Precip Type Encoded
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251,15.8263,0,1015.13,Partly cloudy throughout the day.,22,31,0
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259,15.8263,0,1015.63,Partly cloudy throughout the day.,23,31,0
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204,14.9569,0,1015.94,Partly cloudy throughout the day.,0,1,0
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269,15.8263,0,1016.41,Partly cloudy throughout the day.,1,1,0
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259,15.8263,0,1016.51,Partly cloudy throughout the day.,2,1,0


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = DecisionTreeClassifier(random_state=42, max_depth=5)  # limit tree depth to avoid overfitting
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

print("Train accuracies:", train_accuracies)
print("Test accuracies:", test_accuracies)
print(f"Average Train Accuracy: {np.mean(train_accuracies):.4f}")
print(f"Average Test Accuracy: {np.mean(test_accuracies):.4f}")

Train accuracies: [0.9962935123506389, 0.9959695186749955, 0.996189834374433, 0.9962546816479401, 0.9961250858572114]
Test accuracies: [0.9955938002177182, 0.9969934166191489, 0.9957493131512104, 0.9955935717988595, 0.9961119751166407]
Average Train Accuracy: 0.9962
Average Test Accuracy: 0.9960


In [17]:
import numpy as np

# Calculate bias and variance from accuracies
# Bias is typically related to training error (1 - accuracy)
# Variance can be estimated by the variance of the test accuracies across folds
bias = 1 - np.mean(train_accuracies)
variance = np.var(test_accuracies)

print(f"Estimated Bias: {bias:.4f}")
print(f"Estimated Variance: {variance:.6f}")

Estimated Bias: 0.0038
Estimated Variance: 0.000000


In [18]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []

for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = DecisionTreeClassifier(max_depth=5, random_state=42)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

train_accuracies = np.array(train_accuracies)
test_accuracies = np.array(test_accuracies)


bias = 1 - train_accuracies.mean()

variance = test_accuracies.var()

print(f"Estimated Bias: {bias:.4f}")
print(f"Estimated Variance: {variance:.6f}")

Estimated Bias: 0.0038
Estimated Variance: 0.000000
